In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import glob

In [ ]:
api_key = os.getenv("OPENAI_API")
if api_key and api_key[:3] == "sk-":
    print("API Key is Set")
else:
    print("Something Went wrong")

In [ ]:
MODEL = "gpt-4o-mini"
openai = OpenAI(api_key = api_key)

In [ ]:
context = {}
employees = glob.glob("knowledge-base/employees/*")
# print(employees)
for employee in employees:
    name = employee.split(' ')[-1][:-3]
    doc = ""
    with open(employee,"r") as f:
        doc = f.read()
    context[name]=doc

In [ ]:
len(context.keys())

In [ ]:
products = glob.glob("knowledge-base/products/*")
for product in products:
    name = product.split('\\')[-1][:-3]
    doc = ""
    with open(product,'r') as f:
        doc = f.read()
    context[name] = doc

In [ ]:
context.keys()

In [ ]:
system_message = \
"""
You are an expert in answering accurate questions about Insurellm, the Insurance Texh Company.
Give brief, accurate answers. If you don't know the answer, say that I don't know.
Do not make anything up if you haven't been provided with relevant context.
"""

In [ ]:
def get_relevant_context(message):
    relevant_context = []
    for context_title,context_details in context.items():
        if context_title in message:
            relevant_context.append(context_details)
    return relevant_context

In [ ]:
# print(get_relevant_context('Who is Avery Lancaster'))

In [ ]:
def add_context(message):
    relevane_context = get_relevant_context(message=message)
    if relevane_context:
        message += "\n\nThe following additional context might be relevant in answering this question:\n\n"
        for relevant in relevane_context:
            message += relevant + "\n\n"
    return message

In [ ]:
print(add_context("Who is Avery Lancaster and what is Carllm"))

In [ ]:
def chat(message,history):
    conversation = [
        {"role":"system","content":system_message},
    ]
    for user_msg,assistant_msg in history:
        conversation.append({"role":"user","content":user_msg})
        conversation.append({"role":"assistant","content":assistant_msg})
    message = add_context(message)
    conversation.append({"role":"user","content":message})
    
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=conversation,
        stream=True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat).launch(inbrowser=True)

In [ ]:
get_relevant_context("Carllm")